In [14]:
###
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

###
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing

###
import multiprocessing
import warnings
import gc
import datetime
import graphviz
from time import time
from tqdm import tqdm
from tqdm import tqdm_notebook

In [16]:
# tqdm 적용
for ii in tqdm(range(100)):
	# print 문이 for문 안에 있으면 상태바가 새로 출력됨.
    print(ii)
	# sleep 없으면 너무 빨리 지나가서 상태 확인용
    time.sleep(0.1)

  0%|                                                                                                  | 0/100 [00:00<?, ?it/s]

0


AttributeError: 'builtin_function_or_method' object has no attribute 'sleep'

In [2]:
# data_path = '../../data/extracted/'

In [3]:
# load data
# train
train = pd.read_csv('../../data/extracted/train_final(knn).csv', index_col='TransactionID')

# test
test = pd.read_csv('../../data/extracted/test.csv', index_col='TransactionID')
train.head()
test.head()

,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,...,id-31,id-32,id-33,id-34,id-35,id-36,id-37,id-38,DeviceType,DeviceInfo
TransactionID,,,,,,,,,,,,,,,,,,,,,
3663549,18403224,31.95,W,10409,111.0,150.0,visa,226.0,debit,170.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3663550,18403263,49.00,W,4272,111.0,150.0,visa,226.0,debit,299.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3663551,18403310,171.00,W,4476,574.0,150.0,visa,226.0,debit,472.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3663552,18403310,284.95,W,10989,360.0,150.0,visa,166.0,debit,205.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3663553,18403317,67.95,W,18018,452.0,150.0,mastercard,117.0,debit,264.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
train = reduce_mem_usage(train)
test = reduce_mem_usage(test)

NameError: name 'reduce_mem_usage' is not defined

In [5]:
def reduce_mem_usage(memory_df,verbose=True):
    numerics = ['int16','int32','int64','float16','float32','float64']
    start_mem = memory_df.memory_usage().sum()/1024**2
    for col in memory_df.columns:
        col_type = memory_df[col].dtypes
        if col_type in numerics:
            c_min = memory_df[col].min()
            c_max = memory_df[col].max()
            if str(col_type)[:3]=='int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    memory_df[col] = memory_df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    memory_df[col] = memory_df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    memory_df[col] = memory_df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    memory_df[col] = memory_df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    memory_df[col] = memory_df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    memory_df[col] = memory_df[col].astype(np.float32)
                else: 
                    memory_df[col] = memory_df[col].astype(np.float64)
    end_mem = memory_df.memory_usage().sum()/1024**2
    if verbose : print('Meme usage decrease to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100*(start_mem - end_mem)/start_mem))
    return memory_df

In [6]:
for f in test.columns:
    if test[f].dtype == 'object':
        le = preprocessing.LabelEncoder()
        le.fit(list(test[f].values))
        test[f] = le.transform(list(test[f].values))
    else :
        pass

In [7]:
test_column_list= list(test)
test_column_list

test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 506691 entries, 3663549 to 4170239
Columns: 432 entries, TransactionDT to DeviceInfo
dtypes: float64(399), int64(33)
memory usage: 1.6 GB


In [8]:
test_v_df = test.loc[:,'V1':'V339']
test_v_df

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
TransactionID,,,,,,,,,,,,,,,,,,,,,
3663549,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3663550,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3663551,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3663552,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3663553,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4170235,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4170236,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4170237,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
v_list = list(test_v_df)

In [10]:
test_drop_v_df = test.drop(v_list, axis=1)
test_drop_v_df

,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,...,id-31,id-32,id-33,id-34,id-35,id-36,id-37,id-38,DeviceType,DeviceInfo
TransactionID,,,,,,,,,,,,,,,,,,,,,
3663549,18403224,31.950,4,10409,111.0,150.0,4,226.0,2,170.0,...,107,NaN,390,2,2,2,2,2,2,2184
3663550,18403263,49.000,4,4272,111.0,150.0,4,226.0,2,299.0,...,107,NaN,390,2,2,2,2,2,2,2184
3663551,18403310,171.000,4,4476,574.0,150.0,4,226.0,2,472.0,...,107,NaN,390,2,2,2,2,2,2,2184
3663552,18403310,284.950,4,10989,360.0,150.0,4,166.0,2,205.0,...,107,NaN,390,2,2,2,2,2,2,2184
3663553,18403317,67.950,4,18018,452.0,150.0,2,117.0,2,264.0,...,107,NaN,390,2,2,2,2,2,2,2184
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4170235,34214279,94.679,0,13832,375.0,185.0,2,224.0,2,284.0,...,107,NaN,390,2,2,2,2,2,2,2184
4170236,34214287,12.173,0,3154,408.0,185.0,2,224.0,2,NaN,...,5,NaN,390,2,0,0,1,0,1,107
4170237,34214326,49.000,4,16661,490.0,150.0,4,226.0,2,327.0,...,107,NaN,390,2,2,2,2,2,2,2184


### PCA 

In [17]:
# PCA about v_columns

## 표준화
test_v_df = test.loc[:,'V1':'V339']
features = test.loc[:,'V1':'V339'].columns

## 표준화 진행
test_v_df = StandardScaler().fit_transform(test_v_df)
test_v_df = pd.DataFrame(test_v_df,columns = features)

In [18]:
test_v_df

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,0.00174,-0.185915,-0.243532,0.349202,0.238576,-0.188191,-0.239959,-0.130514,-0.176301,1.038160,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.00174,-0.185915,-0.243532,0.349202,0.238576,-0.188191,-0.239959,-0.130514,-0.176301,-0.872404,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.00174,-0.185915,-0.243532,0.349202,0.238576,-0.188191,-0.239959,-0.130514,-0.176301,-0.872404,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.00174,-0.185915,-0.243532,0.349202,0.238576,-0.188191,-0.239959,-0.130514,-0.176301,1.038160,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.00174,-0.185915,-0.243532,0.349202,0.238576,-0.188191,-0.239959,-0.130514,-0.176301,1.038160,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506686,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
506687,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
506688,0.00174,-0.185915,-0.243532,0.349202,0.238576,-0.188191,-0.239959,-0.130514,-0.176301,-0.872404,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
506689,0.00174,-0.185915,-0.243532,0.349202,0.238576,-0.188191,-0.239959,-0.130514,-0.176301,-0.872404,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
test_v_df = test_v_df.fillna(-999)

In [20]:
columns = ['PCA_V1','PCA_V2','PCA_V3','PCA_V4','PCA_V5','PCA_V6','PCA_V7','PCA_V8','PCA_V9','PCA_V10','PCA_V11','PCA_V12','PCA_V13','PCA_V14','PCA_V15','PCA_V16','PCA_V17','PCA_V18','PCA_V19','PCA_V20','PCA_V21','PCA_V22','PCA_V23','PCA_V24','PCA_V25','PCA_V26','PCA_V27','PCA_V28','PCA_V29','PCA_V30']

In [21]:
# PCA 진행
pca = PCA(n_components=30)
principalComponents = pca.fit_transform(test_v_df)
principalDF = pd.DataFrame(data=principalComponents,columns=columns)

In [22]:
principalDF

,PCA_V1,PCA_V2,PCA_V3,PCA_V4,PCA_V5,PCA_V6,PCA_V7,PCA_V8,PCA_V9,PCA_V10,...,PCA_V21,PCA_V22,PCA_V23,PCA_V24,PCA_V25,PCA_V26,PCA_V27,PCA_V28,PCA_V29,PCA_V30
0,-3163.893811,-189.066477,-542.423908,224.850486,-320.374933,-141.984164,-1.659953,-25.374353,-3.488927,0.880005,...,-0.346870,0.255713,-0.567985,-1.008177,0.340030,0.653385,2.624763,-0.321743,0.060100,-2.049882
1,-3163.925746,-190.208526,-542.738324,220.450904,-319.033331,-141.357188,-1.560588,-25.419952,-4.948940,3.587009,...,-0.269894,0.024978,-0.860660,-0.069204,0.433352,0.557653,0.710092,0.213601,0.719711,0.431297
2,-3164.026023,-190.671321,-543.060578,219.227177,-318.905241,-141.366277,-1.536287,-25.699858,-5.736415,2.977440,...,-1.057479,-0.969825,-0.880546,0.192630,-0.386520,0.085596,0.229679,-0.349135,0.533318,1.014458
3,-3164.433411,-193.490048,-547.041382,220.659614,-320.013098,-141.975686,-1.473717,-29.152477,-4.417083,3.948843,...,0.319288,0.710335,-0.059495,-0.185757,-0.326566,-0.874409,-1.200302,1.498295,-1.163887,0.337567
4,-3164.384973,-191.253915,-543.947332,219.006560,-319.757718,-142.009153,-1.538258,-27.748111,-4.405550,3.930379,...,1.310661,2.422309,-0.829416,0.397504,0.621071,2.530358,-3.033955,-0.092513,-1.539619,-1.371109
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506686,-2288.860863,-527.895476,-351.693455,478.564333,2179.103604,1781.178649,229.629954,-34.179737,0.616041,-6.078376,...,-0.254553,-1.943317,-0.198527,2.504621,0.426986,0.614395,3.316598,-0.962127,-2.571133,0.247301
506687,7090.878908,-3657.304554,1748.418148,60.996095,-474.332293,310.733899,-213.969065,-67.748453,3.368721,18.483473,...,0.521958,0.361748,-0.364542,0.060155,0.421551,0.058592,-0.199416,0.890889,-0.666437,-0.384809
506688,-3163.964193,-190.371068,-542.850503,220.019202,-318.989915,-141.358759,-1.542837,-25.875599,-4.446967,4.015698,...,-0.561422,-1.235429,-0.071959,-0.432600,-0.291238,-0.410967,-0.728538,0.326617,0.318210,-0.087090
506689,-3163.964193,-190.371068,-542.850503,220.019202,-318.989915,-141.358759,-1.542837,-25.875599,-4.446967,4.015698,...,-0.561422,-1.235429,-0.071959,-0.432600,-0.291238,-0.410967,-0.728538,0.326617,0.318210,-0.087090


### KNN

In [23]:
# KNN about test_drop_v_df columns

## 표준화
test_drop_v_df = test.drop(v_list, axis=1)
features = test_drop_v_df.columns

## 표준화 진행
test_drop_v_df = StandardScaler().fit_transform(test_drop_v_df)
test_drop_v_df = pd.DataFrame(test_drop_v_df,columns = features)

In [24]:
imputer = KNNImputer(n_neighbors=5)

In [ ]:
imputed = imputer.fit_transform(test_drop_v_df)
df_imputed = pd.DataFrame(imputed, columns = test_drop_v_df.columns)
df_imputed.info()

### DATA Concatenate

In [ ]:
df_13_axis1 = pd.concat([df_1, df_3], axis=1) # column bind